In [962]:
import numpy as np
from scipy.spatial.transform import Rotation as sciptRot

In [963]:
l = 3
k = 1
q = [10, 10, 10]
Q = np.diag(np.hstack([np.diag(q[2]*np.eye(3)), np.diag(q[1]*np.eye(3)), np.diag(q[0]*np.eye(3))]))
P_0 = np.diag(np.hstack([np.diag(np.eye(3)), np.diag(100*np.eye(3))]))
V = np.diag(np.hstack([np.diag(0.1*np.eye(3)), np.diag(np.eye(3))]))
p_0 = np.array([2.5, 0, 10]).T
r_0 = np.eye(3)
p_hat_0 = np.array([-2, 4,3]).T
# w, x, y, z
Lambda_bar_0 = np.array([np.sqrt(2)/2, np.sqrt(2)/2, 0, 0]).T

In [964]:
# time 
times = np.linspace(0,100,9)
# source point
z = np.array([[0, 0, 0], [5, 0, 0], [2.5, 2.5, 0]])

#C
#position
p_list = []
#velocity
v_list = []
#angular velocity
omega_list = []

#velocity in different frame, R.T*v
v_bar_list = []
#position in different frame, R.T*p
p_bar_list = []
#position_bar_hat
p_bar_hat_list = []
#orientation list
Lambda_bar_list = []
#rot mat list
r_hat_list = []
# Racaati
P_list = []
P_list.append(P_0)

A_list = []
C_list = []


In [965]:
p_list = []
r_list = []
p_hat_list = []
Lambda_bar_list = []
p_hat_list.append(p_hat_0)

Lambda_bar_list.append(Lambda_bar_0)
#USE VSTACK AFTERWARD
# inital p and R definition

In [966]:
def make_S(content):
    output = [[0,           -content[2],    content[1]],
              [content[2],  0,              -content[0]],
              [-content[1], content[0],     0]]
    return np.array(output)

In [967]:
def make_d(R, p, z):
    output = (np.matmul(R.T, ((p - z)/np.linalg.norm(p-z)))).tolist()
    return np.array(output).reshape((3,1))

In [968]:
new_estimate = []
r_list.append(r_0)
p_bar_hat = np.matmul(r_list, p_hat_list[-1])
p_bar_hat_list.append(p_bar_hat)

r_hat = sciptRot.from_quat(Lambda_bar_list[-1]).as_matrix()

r_hat_list.append(r_hat)
for t in times:
    print(t)
    # update p, v, omega, , orientation(r)
    p = np.array([2.5 + 2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    v = np.array([-np.sin(0.4*t), np.cos(0.4*t), 0.6*t])
    omega = np.array([0.1*np.sin(t), 0.4*np.cos(2*t), 0.6*t])
    v_bar = np.matmul(r_list, v)[0]
    p_bar = np.matmul(r_list, p)

    A1 = np.hstack([-make_S(omega), np.zeros((3,3))])
    A2 = np.hstack([np.zeros((3,3)), -make_S(omega)])
    A = np.vstack([A1, A2])
    A_list.append(A)
    C = []
    for idx in range(l):
        base = (np.eye(3)- np.matmul(make_d(r_list[-1], p, z[idx]),make_d(r_list[-1], p, z[idx]).T))
        temp = np.matmul(base, make_S(np.matmul(np.array(r_hat).T, z[idx])))
        C.append(np.hstack((temp, base)).reshape(3,6))
        print(C)
        break
    C = np.array(C)
    C_list.append(C)
    

    p_list.append(p)
    v_list.append(v)
    omega_list.append(omega)
    v_bar_list.append(v_bar)
    p_bar_list.append(p_bar)
    first_part = np.hstack((omega, -np.matmul(make_S(omega),p_bar_hat.T).T[0] + v_bar)).reshape((-1,1))
    # print("first_part", first_part)

    upper = np.zeros(3)
    lower = np.zeros(3)
    for idx in range(l):
        upper = q[idx]*(np.matmul(make_S(np.matmul(np.array(r_hat).T, z[idx])), np.matmul((np.eye(3)-np.matmul(make_d(r_list[-1], p, z[idx]), make_d(r_list[-1], p, z[idx]).T)), (p_bar_hat -np.matmul(np.array(r_hat).T, z[idx])).T)))
        lower = q[idx]*(np.matmul((np.eye(3)-np.matmul(make_d(r_list[-1], p, z[idx]), make_d(r_list[-1], p, z[idx]).T)), (p_bar_hat -np.matmul(np.array(r_hat).T, z[idx])).T))
        break
    second_part = np.matmul(k*P_list[-1], np.vstack([upper, lower]))
    result = first_part - second_part
    omega_hat = result[0:3]
    p_bar_hat = result[3:]
    p_bar_hat_list.append(p_bar_hat)

    r_hat = np.matmul(np.array(r_hat), make_S(omega_hat))
    r_hat_list.append(r_hat)
    print(np.shape(C))
    print(np.matmul(C,P_list[-1]))
    print(np.matmul(P_list[-1], np.matmul(C.T, np.matmul(Q,))))
    P_dot = np.matmul(A,P_list[-1]) + np.matmul(P_list[-1], A.T) - np.matmul(P_list[-1], np.matmul(C.T, np.matmul(Q,np.matmul(C,P_list[-1])))) + V
    P_list.append(P_dot)
    break


0.0
[[[ 0.          0.          0.          0.8         0.
   -0.4       ]
  [ 0.          0.          0.          0.          1.
    0.        ]
  [ 0.          0.          0.         -0.4         0.
    0.2       ]]

 [[ 0.          0.          5.          1.          0.
    0.        ]
  [ 0.          0.          0.          0.          1.
    0.        ]
  [ 0.          0.          0.          0.          0.
    0.        ]]

 [[ 0.55555556 -0.55555556  2.22222222  0.94444444  0.05555556
   -0.22222222]
  [-0.55555556  0.55555556 -2.22222222  0.05555556  0.94444444
    0.22222222]
  [-0.27777778  0.27777778 -1.11111111 -0.22222222  0.22222222
    0.11111111]]]
(3, 3, 6)
[[[  0.           0.           0.          80.           0.
   -40.        ]
  [  0.           0.           0.           0.         100.
     0.        ]
  [  0.           0.           0.         -40.           0.
    20.        ]]

 [[  0.           0.           5.         100.           0.
     0.        ]
  [  0.

/tmp/ipykernel_37135/2187250428.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(output)


TypeError: matmul() takes from 2 to 3 positional arguments but 1 were given

array([[0],
       [1],
       [2]])